***Airline ticket assistant***

***Import Libraries***

In [8]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

***Load environment***

In [9]:
# load_dotenv(override=True)

# openai_api_key = os.getenv('OPENAI_API_KEY')
# if openai_api_key:
#     print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
# else:
#     print("OpenAI API Key not set")
    
# MODEL = "gpt-4o-mini"
# openai = OpenAI()

# As an alternative, if you'd like to use Ollama instead of OpenAI
# Check that Ollama is running for you locally (see week1/day2 exercise) then uncomment these next 2 lines
MODEL = "llama3.2"
openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')


***Prompts setup***

In [10]:
system_message = "You are a helpful assistant for an Airline called Flighty. "
system_message += "Give short, sarcastic answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, please say I dont know."

***Gradio UI, along with chat function in the format needed by Gradio***

In [12]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch(share=True)

* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://b578ca082f38835f96.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


***Function & Tools***

In [13]:
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [14]:
get_ticket_price("Berlin")

Tool get_ticket_price called for Berlin


'$499'

This code defines a Python dictionary named price_function. This dictionary is structured in a way that is commonly used to describe a function for a language model to call, often referred to as ***"tool calling" or "function calling"***.

Here's a breakdown of the key-value pairs in the dictionary:

"name": "get_ticket_price": This specifies the name of the function that the language model should call.
"description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'": This provides a clear description of what the function does and when the language model should consider calling it. This helps the model understand the purpose of the tool.
"parameters": { ... }: This nested dictionary describes the parameters that the get_ticket_price function expects.
"type": "object": Indicates that the parameters are structured as an object (in this case, a JSON object).
"properties": { ... }: This nested dictionary lists the individual parameters of the function.
"destination_city": { ... }: This describes a single parameter named destination_city.
"type": "string": Specifies that the destination_city parameter should be a string.
"description": "The city that the customer wants to travel to": Provides a description of what the destination_city parameter represents.
"required": ["destination_city"]: This list specifies which of the defined properties are required. In this case, destination_city is a required parameter.
"additionalProperties": False: This indicates that no additional parameters beyond those explicitly listed in properties are allowed.
In summary, the price_function dictionary is a structured definition of a function that a language model can use to get the price of a ticket. It tells the model the function's name, what it does, what information it needs (destination_city), and that this information is required. This allows the language model to understand how and when to use this tool to fulfill a user's request.

In [15]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [16]:
tools = [{"type": "function", "function": price_function}]

***Chat function for interaction with LLM***

In [17]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools) #tools added to chat function

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

***Tool call function***

This Python function handle_tool_call is designed to process a specific type of message received from a language model that indicates the model wants to call a tool (or function).

Here's a breakdown:

def handle_tool_call(message):: Defines the function handle_tool_call that takes one argument, message, which is expected to be a message object from the language model's response.
tool_call = message.tool_calls[0]: Accesses the first tool call in the tool_calls list within the message object. A message from a language model might contain a list of tool calls if the model decides to use one or more tools in response to the user's input. This line assumes there is at least one tool call and takes the first one.
arguments = json.loads(tool_call.function.arguments): Extracts the arguments that the language model wants to pass to the function.
tool_call.function.arguments: This accesses the arguments provided by the model for the function call. These arguments are typically in a JSON string format.
json.loads(...): Parses the JSON string into a Python dictionary, making it easy to access the individual arguments by their keys.
city = arguments.get('destination_city'): Retrieves the value associated with the key 'destination_city' from the arguments dictionary. The .get() method is used here, which is safer than direct dictionary access (arguments['destination_city']) because it will return None if the key doesn't exist, preventing a KeyError.
price = get_ticket_price(city): Calls another function, get_ticket_price, passing the extracted city as an argument. This is where the actual work of getting the ticket price happens. The result is stored in the price variable.
response = { ... }: Constructs a dictionary representing the response that needs to be sent back to the language model after the tool call is executed. This response informs the model about the result of the tool call.
"role": "tool": Indicates that this message is from a "tool".
"content": json.dumps({"destination_city": city,"price": price}): Provides the result of the tool call. It's a JSON string containing the destination_city and the fetched price. This information is what the language model will use to formulate its final response to the user.
"tool_call_id": tool_call.id: Includes the ID of the tool call that this response corresponds to. This helps the language model match the tool call response to the original tool call request.
return response, city: The function returns two values: the response dictionary (to be sent back to the language model) and the extracted city. Returning the city might be useful for subsequent steps in the conversation flow.
In summary, the handle_tool_call function acts as an intermediary when a language model wants to use a predefined tool (like getting a ticket price). It parses the model's request, executes the corresponding function (get_ticket_price), and formats the result in a way that the language model can understand and use to generate a final user-facing response.

In [18]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}),
        "tool_call_id": tool_call.id
    }
    return response, city

In [19]:
gr.ChatInterface(fn=chat, type="messages").launch(share=True)

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


Tool get_ticket_price called for 
Tool get_ticket_price called for London
Tool get_ticket_price called for London
Tool get_ticket_price called for London
Tool get_ticket_price called for London
Tool get_ticket_price called for London
Tool get_ticket_price called for London
Tool get_ticket_price called for Berlin
